<a href="https://colab.research.google.com/github/devilskw/tech_challenge_fase2/blob/main/techchallenge_fase2_grupo16_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Nota
Criarmos a classe **MelhorTrajeto** para organização da aplicação do projeto, porém, vamos altear para deixar aqui de uma forma que fique fácil o entendimento de como foi pensado o algoritmo genético, analises e mostrar resultados obtidos de forma descrita



## Pré-definições e Bibliotecas

Vamos usar a biblioteca random, para nos apoiar quando precisarmos gerar aleatoriedades

Também vamos definir para o exemplo alguns valores padrões a serem utilizados

```python
tamanho_populacao: int=100, # quantidade de indivíduos a serem gerados na população
qtde_geracoes: int=50, # quantidade de gerações (condição de término)
qtde_amostra_selecao: int=10, # quantidade de amostras que serão separadas para a seleção
probabilidade_mutacao: float=0.10) # probabilidade para apoiar na decisão de realizar (ou não) a mutação
```

In [41]:
import random

tamanho_populacao: int=100
qtde_geracoes: int=50
qtde_amostra_selecao: int=10
probabilidade_mutacao: float=0.10

Vamos nos basear sempre no fluxo mostrado em aula para entendermos melhor o que cada parte do código representa no fluxo algoritmo genético:

[Clique aqui para ver a imagem do fluxograma](https://github.com/devilskw/tech_challenge_fase2/blob/main/assets/fluxograma_algorimo_genetico.png)

# 1. Gerar a população inicial

A população inicial será gerada através da criação de rotas aleatórias baseados nos pontos de entrega que deverão ser efetuadas, até o máximo definido para o tmanho da população.

Criamos o método **\_\_gerar_rota_aleatoria\_\_** para apoiar na geração da população.

Este método recebe como parâmetro de entrada aos pontos de **entregas** do dia, cria uma lista somente com os identificador único dos clientes e realiza o "embaralhamento" da ordem, retornando essa lista como uma nova rota de ordem aleatória.

In [43]:
def __gerar_rota_aleatoria__(entregas):
  rota = list(entregas.keys())
  random.shuffle(rota)
  return rota

Para apoiar na geração da população temos o método  **\_\_gerar_populacao\_\_**, que receberá como parâmetro os pontos de entrega (**pntos_entrega**) e a quantidade de indivíduos que devem ser gerados para a população (**tamanho_populacao**)


Obs.: Ficou definido que, na população, não seria adicionada na rota a localização do centro de distribuição (CD), porém, ela será considerada posteriormente, na etapa de **Seleção**.

In [52]:
def __gerar_populacao__(entregas, tamanho_populacao) -> list[str]:
  return [(__gerar_rota_aleatoria__(entregas), 0) for _ in range(tamanho_populacao)]

# 2. Avaliar a aptidão dos indivíduos (fitness)

Para avaliarmos a aptidão dos indivíduos, vamos considerar o cálculo da distância total de cada rota da população, lembrando que, para diferenciar neste projeto, será considerado como ponto inicial a localização do centro de distribuição, que será onde o motorista deverá iniciar a partida para as suas entregas.


O método **"\_\_calcular_fitness\_\_"** será o nosso cálculo de ***'fitness'***, ou seja, será o cálculo a ser considerado para avaliar a aptidão dos indivíduos no algoritmo genético.


#### Mas, como será o nosso cálculo de aptidão?

Vamos considerar como cálculo a distância total da rota:

Ele receberá a posição do centro de ditribuição (CD), os dados de rotas/entregas.

Ele deverá calcular a distância total que será percorrida pelo motorista considerando a rota informada (seguindo fielmente a ordenação passada).

Para isso, temos o método **"\_\_calcular_distancia_total_rota\_\_"**.

Para calcular a distância entre o ponto de localização inicial e final, iremos utilizar o método **"\_\_calcular_distancia_pontos\_\_"**, que nada mais é que um cálculo matemático que mede a distância entre as duas localizações. Pensando em um plano cartesiano, imagine que cada endereço (e CD) possui uma localização (X,Y). Então, pensando no ponto inicial (X', Y') e ponto final (X'', Y''), o calculo seria:

1. subtrair X''  por X'', e depois elevar ao quadrado. Vamos considerar que esse resultado será R1
2. subtrair Y''  por Y'', e depois elevar ao quadrado. Vamos considerar que esse resultado será R2
3. Somar R1 e R2, resultando em R3
4. O resultado final (distância) será a raiz quadrada de R3

Inicialmente, ele irá calcular qual a distância entre o CD e o primeiro endereço de entrega.

Na ordem da rota, vai calculando a distância do ponto anterior com o próximo ponto de entrega, e somando esse resultado para obter a distância total

In [45]:
def __calcular_distancia_pontos__(ponto1: tuple[float, float], ponto2: tuple[float, float]):
    return ((ponto2[0] - ponto1[0])**2 + (ponto2[1] - ponto1[1])**2) ** 0.5

def __calcular_distancia_total_rota__(centro_distribuicao, rota, entregas):
  distancia_Total = 0
  if len(rota) > 0:
    distancia_Total += __calcular_distancia_pontos__(centro_distribuicao, entregas[rota[0]])
    for i in range(len(rota) - 1):
      endereco1 = rota[i]
      endereco2 = rota[i + 1]
      distancia_Total += __calcular_distancia_pontos__(entregas[endereco1], entregas[endereco2])
  return distancia_Total

def __calcular_fitness__(centro_distribuicao, entregas, populacao):
  populacao_tmp = populacao
  for ix in range(len(populacao)):
      populacao_tmp[ix] = (populacao[ix][0], __calcular_distancia_total_rota__(centro_distribuicao, populacao[ix][0], entregas))
  return populacao_tmp

# 3. Verificar a condição de término

A condição de término será baseada na limitação de iterações possíveis para as gerações, que será controalada pela variável **"qtde_geracoes"** (que foi citada inicialmente)

Mais para frente, no código, esta etapa será representada pelo ciclo:

```python
for _ in range(qtde_geracoes):
```


# 4. Seleção

Com base num valor definido vamos pegar uma amostragem da população e considerar o menor valor encontrado, selecionando 2 indivíduos a cada passagem.

### Como funciona?

O método **"\_\_realizar_selecao\_\_"** será o responsável pela seleção, onde receberá como parâmetro a população, e também a quantidade definida para amostrar da seleção.

Neste método, vamos utilizar o método **"sample"** da biblioteca **"random"**, que será o responsável por pegar essa amostragem.

A partir dessa amostragem, o método irá retornar o indivíduo identificado com menor **"fitness"**, ou seja, com **menor** valor de distância total da rota.

No caso do nosso projeto, vamos gerar 2 indivíduos (**individuo1** e **individuo2**), que apoiarrá para a próxima etapa, que é a etapa de cruzamento.


In [46]:
def __realizar_selecao__( populacao, qtde_amostra_selecao):
  amostragem = random.sample(populacao, qtde_amostra_selecao)
  return min(amostragem, key=lambda x: x[1])

## 5. Cruzamento (Crossover) e 6. Mutação

### No cruzamento

Para o cruzamento, vamos considerar de forma aleatória um índice para realizar o cruzamento entre os indivíduos.

O método que realizará o cruzamento será o **"\_\_realizar_cruzamento\_\_"**, que receberá entregas, individuo1 e individuo2.

Então, ao receber os 2 indivíduos gerados a partir da etapa de ***seleção***, a idéia é que ele realize o cruzamento deles, gerando um ***descentente***.

Para fazer esse cruzamento, vamos aleatoriamente gerar um índice de cruzamento, que irá definir até que ponto irá pegar os pontos de entrega do **indivíduo1** (na ordem), e depois complementar o restante com o **indivíduo2**, lembrando que não podemos repetir um mesmo endereço de entrega.


### Na mutação

Para a mutação, haverá uma condição para realizar ou não a mutação. Para este projeto, vamos realizar a mutação na parte parcial considerada do cruzamento
 para o **individuo2**.

 Neste caso, temos o método **"\_\_realizar_mutacao\_\_"**, que receberá como parâmetros a lista com os pontos de entrega parciais que acabamos (do **individuo2**) e o valor definido para a probabilidade para apoiar na realização ou não da mutação.

#### Para definir se irá realizar ou não a mutação, o que fizemos:

  - Usamos o método **"random()"** da biblioteca **"random"**, que gera um valor aleatório;
  - Verificamos se esse valor aleatório é menor qua a probabilidade para realizar a mutação. Se menor, a condição será true, senão false;
  - Se a condição anterior for true, ainda vamos considerar **somente** quando a lista de pontos de entrega parciais tiver, pelo menos 2 pontos de entrega. Então, se também tiver 2 ou mais pontos de entrega na lista, ele realizará a mutação, senão não

#### Se for para realizar a mutação:
 - Geramos aleatoriamente um índice (**indice_mutacao**) para trocar 2 pontos de entrega.
 - Baseado nesse índice de mutação, fazemos a troca do ponto de entrega na posição do índice com o da próxima posição (por isso que subtraímos com **-2**, para facilitar a evitar erro por tentar pegar item de uma lista com índice acima do possível)
 - Devolve os pontos de entrega parciais após essas alterações

#### Se não for para realizar a mutação:
 - Simplesmente retorna os pontos de entrega parciais como vieram



In [47]:
def __realizar_mutacao__(pontos_entrega_parciais: list, probabilidade_mutacao: float):
  mutantes = pontos_entrega_parciais
  if random.random() < probabilidade_mutacao and len(pontos_entrega_parciais) >= 2:
    indice_mutacao = random.randint(0, len(pontos_entrega_parciais) - 2)
    mutante_tmp = mutantes[indice_mutacao]
    mutantes[indice_mutacao] = mutantes[indice_mutacao +1]
    mutantes[indice_mutacao+1] = mutante_tmp
  return mutantes

def __realizar_cruzamento__(entregas, individuo1, individuo2):
  indice_cruzamento = random.randint(1, len(entregas) - 1)
  descendente = individuo1[0][:indice_cruzamento]
  descendente += __realizar_mutacao__([endereco for endereco in individuo2[0] if endereco not in descendente], probabilidade_mutacao)
  return descendente

## 7. Substituir População Antiga
A cada geração, uma nova população será formada com base nos decendentes criados com a mesma quantidade definida para população

## O Algoritmo Genético

O algoritmo genético será representado pelo método **"gerar_melhor_trajeto"**, que utilizará os métodos explicados anteriormente.

Então, ao vermos o método, vemos que ele irá seguir conforme o fluxograma apresentado para algoritmo genético.

In [53]:
  def gerar_melhor_trajeto( centro_distribuicao: tuple[float, float], entregas: dict[str, tuple[float, float]]):
    populacao = __gerar_populacao__(entregas,tamanho_populacao)
    for _ in range(qtde_geracoes):
      populacao = __calcular_fitness__(centro_distribuicao, entregas, populacao)
      populacao_temp = []
      for _ in range(tamanho_populacao):
        individuo1 = __realizar_selecao__(populacao, qtde_amostra_selecao)
        individuo2 = __realizar_selecao__(populacao, qtde_amostra_selecao)
        descendente = __realizar_cruzamento__(entregas, individuo1, individuo2)
        populacao_temp.append((descendente, __calcular_distancia_total_rota__(centro_distribuicao, descendente, entregas)))
      populacao = populacao_temp
    melhor_trajeto = min(populacao, key=lambda x: x[1])
    return melhor_trajeto

## Dados de exemplo para demonstração e testes

Vamos agora criar um exemplo para demonstrarmos o algoritmo, analisando os resultados também

Vamos primeiro definir onde ficará o centro de distribuição

In [54]:
localizacao_centro_distribuicao = (3.13, 1.44)

Agora, vamos definir os locais em que o motorista precisará realizar as entregas de cada cliente

In [55]:
entregas_motorista = {
    'Cliente A': (0.11, 0.22),
    'Cliente B': (1.11, 2.22),
    'Cliente C': (3.11, 1.22),
    'Cliente D': (5.11, 3.22),
    'Cliente E': (2.11, 4.22)
}

Agora, vamos chamar o nosso algoritmo genético e verificarmos os resultados

In [58]:
trajeto = gerar_melhor_trajeto(localizacao_centro_distribuicao, entregas_motorista)

print("Melhor trajeto:", trajeto[0])
print("Total Distance:", round(trajeto[1], 2))

Melhor trajeto: ['Cliente C', 'Cliente D', 'Cliente E', 'Cliente B', 'Cliente A']
Total Distance: 10.68
